# ByNoemie RAG - Prompt Testing Notebook

This notebook demonstrates:
1. Basic LLM completion
2. Vibe tag generation
3. LangGraph workflows
4. LangSmith tracing

In [ ]:
# Setup
import os
import sys
sys.path.insert(0, '..')

# Set API key (get free key at https://console.groq.com)
os.environ['GROQ_API_KEY'] = 'your_key_here'  # Replace with actual key

# Optional: Enable LangSmith tracing
# os.environ['LANGCHAIN_API_KEY'] = 'your_langsmith_key'
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_PROJECT'] = 'bynoemie-testing'

## 1. Basic LLM Completion

In [ ]:
from src.llm import create_llm_client

# Create client (auto-detects provider)
client = create_llm_client()
print(f"Using: {client.provider_name}")

In [ ]:
# Simple completion
response = client.chat(
    system_prompt="You are a fashion expert.",
    user_prompt="Describe the vibe of a black sequin dress in 5 words."
)

print(f"Response: {response.content}")
print(f"Tokens: {response.usage}")
print(f"Latency: {response.latency_ms:.0f}ms")

## 2. Rule-Based Vibe Extraction

In [ ]:
from src.vibe_generator import extract_vibes_from_product, get_vibe_scores

# Sample product
product = {
    "product_name": "Coco Dress",
    "product_type": "Dress",
    "product_description": "All eyes on you in the Coco Dress, an ultra-mini silhouette covered in oversized black sequins. Featuring slim straps and a daring open back, it's made for nights that sparkle.",
    "colors_available": "Black, Gold",
    "material": "Sequin"
}

# Extract vibes
vibes = extract_vibes_from_product(product)
print(f"Extracted vibes: {vibes}")

# Get scores
scores = get_vibe_scores(product)
print(f"\nTop vibe scores:")
for vibe, score in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"  {vibe}: {score:.2f}")

## 3. LLM-Enhanced Vibe Generation

In [ ]:
from src.llm import parse_json_response

# Free-form prompt
prompt = f"""Generate creative vibe tags for this product:

Product: {product['product_name']}
Description: {product['product_description']}
Colors: {product['colors_available']}

Create 8-10 evocative tags. Be creative and specific.
Return JSON: {{"vibe_tags": [...], "mood_summary": "..."}}"""

response = client.chat(
    system_prompt="You are a creative fashion stylist. Generate memorable, evocative vibe tags.",
    user_prompt=prompt,
    temperature=0.8
)

result = parse_json_response(response.content)
print(f"Vibe tags: {result.get('vibe_tags', [])}")
print(f"Mood: {result.get('mood_summary', 'N/A')}")

## 4. LangGraph Workflow

In [ ]:
from src.vibe_generator import create_vibe_generator

# Create generator with LangGraph workflow
generator = create_vibe_generator()

# Generate vibes
result = generator.generate(
    product_id="test-001",
    product_name=product["product_name"],
    product_type=product["product_type"],
    description=product["product_description"],
    colors=product["colors_available"],
    material=product["material"]
)

print(f"Status: {result['status']}")
print(f"Vibe Tags: {result['vibe_tags']}")
print(f"Mood: {result.get('mood_summary', 'N/A')}")

## 5. LangChain LCEL Chains

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableParallel

# Get LangChain model
llm = client.get_langchain_model()

# Create parallel chains
occasion_chain = ChatPromptTemplate.from_template("What occasion is a {item} best for? One word.") | llm | StrOutputParser()
mood_chain = ChatPromptTemplate.from_template("What mood does a {item} evoke? One word.") | llm | StrOutputParser()

parallel = RunnableParallel(
    occasion=occasion_chain,
    mood=mood_chain
)

result = parallel.invoke({"item": "black sequin dress"})
print(f"Occasion: {result['occasion']}")
print(f"Mood: {result['mood']}")

## 6. Batch Processing

In [ ]:
# Sample products
products = [
    {
        "product_name": "Coco Dress",
        "product_description": "Ultra-mini sequin dress with open back",
        "colors_available": "Black, Gold",
        "material": "Sequin"
    },
    {
        "product_name": "Tiara Satin Dress",
        "product_description": "Feminine with a touch of fantasy",
        "colors_available": "White, Champagne",
        "material": "Silk, Satin"
    },
    {
        "product_name": "Chantelle Heels",
        "product_description": "Classic stiletto heels with elegant lines",
        "colors_available": "Black, Nude",
        "material": "Leather"
    }
]

# Rule-based batch
from src.vibe_generator import process_products_batch

results = process_products_batch(products)

for r in results:
    print(f"\n{r['product_name']}: {r['vibe_tags'][:5]}")

## 7. Compare Methods

In [ ]:
import time

product = products[0]

# Rule-based
start = time.time()
rule_vibes = extract_vibes_from_product(product)
rule_time = (time.time() - start) * 1000

print(f"Rule-based ({rule_time:.0f}ms): {rule_vibes}")

# LLM
start = time.time()
llm_result = generator.generate(
    product_name=product["product_name"],
    description=product["product_description"],
    colors=product["colors_available"],
    material=product["material"]
)
llm_time = (time.time() - start) * 1000

print(f"LLM-based ({llm_time:.0f}ms): {llm_result['vibe_tags']}")

## 8. Save Results

In [ ]:
import json
from pathlib import Path

# Save to data directory
output_dir = Path('../data/outputs')
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / 'notebook_results.json'
with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"Saved to {output_file}")